In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.2f' % x)

#### нужно получить распредение населения по возрастным группам
#### 0-17, 18-60, 61 и старше в разрезе мужчин и женщин. Необходимо прогнозное
#### количество женщин детородного возраста, чтобы по регулируемому показателю 
#### фертильности, получить прогноз всего населения Украины.
#### Далее, зная фертильность и количество детей, можно спрогнозировать расходы
#### которые должно взять на себя государство, чтобы увеличилась рождаемость.

#### try to recover AI's demographic  model

In [3]:
pop=pd.read_excel('data/pop_per_year.xlsx')#с 2012 по 2018 по годам жизни мужчины и женщины

In [4]:
pop2013=pop[pop['рік']==2013]#исходный факт
pop2013.head(2)

,рік,вік,Обидві статі,жінки,чоловіки
81,2013,0,517301,250481,266820
82,2013,1,498371,241404,256967


In [5]:
pop2017=pop[pop['рік']==2017]#исходный факт
pop2017.head(2)

,рік,вік,Обидві статі,жінки,чоловіки
405,2017,0,394626,190833,203793
406,2017,1,408645,197600,211045


In [6]:
#коэффициенты відтворення та міграції по мужчинам и женщинам
#выделены из файлы модели, нужно сделать отдельными сериями,чтобы не привязываться к файлам
koef_vidtvorennya_nas_f=pd.read_excel('D:/Users/o.lukasevych/Documents/Python_Scripts/Ukr_budget/excel/Copy of Демографическая модель изменения населения Украины.xlsx',\
                                     sheet_name='Sheet1')['відтворення населення жінки']
koef_vidtvorennya_nas_m=pd.read_excel('D:/Users/o.lukasevych/Documents/Python_Scripts/Ukr_budget/excel/Copy of Демографическая модель изменения населения Украины.xlsx',\
                                     sheet_name='Sheet1')['відтворення населення чоловіка']
koef_mizhnar_saldo_f=pd.read_excel('D:/Users/o.lukasevych/Documents/Python_Scripts/Ukr_budget/excel/Copy of Демографическая модель изменения населения Украины.xlsx',\
                                     sheet_name='Sheet1')['міжнародне сально жінки']
koef_mizhnar_saldo_m=pd.read_excel('D:/Users/o.lukasevych/Documents/Python_Scripts/Ukr_budget/excel/Copy of Демографическая модель изменения населения Украины.xlsx',\
                                     sheet_name='Sheet1')['міжнародне сальдо чоловіка']

In [7]:
koef_zhyvonar=99.3835113134263 #коэффициент живорожденных от рожденных
koef_mertvonar=0.616488686573717 #коэффициент мертворожденных от родженных
koef_pomer=1.45894635287282 #коэффициент умерших от всего населения

koef_usogo_nar_f=48.3748938322502 #коэффициент всего рожденных женщин
koef_zhivonar_f=48.3876667210801 #коэффициент живорожденных женщин
koef_mertvonar_f=46.3157894736842 #коэффициент мертворожденных женщин
koef_pomer_f=50.9026011137936 #коэффициент умерших женщин

  #коэффициенты по мужчинам
koef_usogo_nar_m=51.6251061677498
koef_zhivonar_m=51.6123332789199
koef_mertvonar_m=53.6842105263158
koef_pomer_m=49.0973988862064

    #коэффициенты миграции
koef_migrant_f=30.6820386779639
koef_migrant_m=69.3179613220361

# на входе функции серия женщин по годам жизни,серия мужчины по годам жизни, всего населения
# кол-во детей на 1000 женщин,коеффициент миграции
def model(f_years,m_years,popTotal,koef_fert,koef_migrant):
    fertilnyh=f_years[18:50].sum()
    narodzhennyh=fertilnyh*koef_fert/1000
    zhivonarodzhennyh=narodzhennyh*koef_zhyvonar/100
    mertvonarodzhennyh=narodzhennyh*koef_mertvonar/100
    pomer=popTotal*koef_pomer/100
    
    all_data=pd.Series({'fertilnyh':fertilnyh,'narodzhennyh':narodzhennyh,\
    'zhivonarodzhennyh':zhivonarodzhennyh,\
    'mertvonarodzhennyh':mertvonarodzhennyh,\
    'pomer':pomer})
    
    #количества по женщинам за текущий год
    usogo_nar_f=narodzhennyh*koef_usogo_nar_f/100 #всего рожденных женщин
    zhivonar_f=koef_zhivonar_f/100*zhivonarodzhennyh #количество живорожденных женщин
    mertvonar_f=koef_mertvonar_f/100*mertvonarodzhennyh #количество мертворожденных женщин
    pomer_f=koef_pomer_f/100*pomer #количество умерших женщин
    
    data_f=pd.Series({'usogo_nar_f':usogo_nar_f,'zhivonar_f':zhivonar_f,\
                      'mertvonar_f':mertvonar_f,'pomer_f':pomer_f})
    
    #количества по мужчинам за текущий год
    zhivonar_m=koef_zhivonar_m/100*zhivonarodzhennyh
    usogo_nar_m=narodzhennyh*koef_usogo_nar_m/100
    mertvonar_m=koef_mertvonar_m/100*mertvonarodzhennyh
    pomer_m=koef_pomer_m/100*pomer
    
    data_m=pd.Series({'usogo_nar_m':usogo_nar_m,'zhivonar_m':zhivonar_m,\
                     'mertvonar_m':mertvonar_m,'pomer_m':pomer_m})
    
    #серии відтворення населення
    vidtvorennya_nas_f=pd.Series(pomer_f*koef_vidtvorennya_nas_f/100*(-1))
    vidtvorennya_nas_m=pd.Series(pomer_m*koef_vidtvorennya_nas_m/100*(-1))
    
    # серии мигрантов
    migrant=popTotal*koef_migrant/100
    migrant_f=migrant*koef_migrant_f/100
    migrant_f_years=pd.Series(koef_mizhnar_saldo_f*migrant_f/100)
    migrant_m=migrant*koef_migrant_m/100
    migrant_m_years=pd.Series(koef_mizhnar_saldo_m*migrant_m/100)
    
    # серии женщин следующего года по годам жизни
    rest1=f_years[:-1]
    rest2=vidtvorennya_nas_f.drop(0).reset_index(drop=True)
    rest3=migrant_f_years.drop(0).reset_index(drop=True)
    rest1=rest1.astype('float64')
    rest4=rest1+rest2+rest3
    rest4=rest4[:-1]
    rest_first=zhivonar_f+vidtvorennya_nas_f[0]+migrant_f_years[0]
    rest_last=pd.Series(f_years.iloc[-1]+f_years.iloc[-2]+\
                vidtvorennya_nas_f.iloc[-1]+\
                migrant_f_years.iloc[-1])
    rest=rest4.append(rest_last).reset_index(drop=True)
    rest.index=rest.index+1
    rest_f=pd.concat([rest, pd.Series(rest_first)])
    female_next=rest_f.sort_index()
    
    rest_series=pd.Series({'rest':rest,'rest1':rest1,\
                           'rest2':rest2,'rest3':rest3,\
                           'rest4':rest4,\
                           'rest_first':rest_first,'rest_last':rest_last})
    
    # серии мужчин следующего года по годам жизни
    mrest1=m_years[:-1]
    mrest2=vidtvorennya_nas_m.drop(0).reset_index(drop=True)
    mrest3=migrant_m_years.drop(0).reset_index(drop=True)
    mrest1=mrest1.astype('float64')
    mrest4=mrest1+mrest2+mrest3
    mrest4=mrest4[:-1]
    mrest_first=zhivonar_m+vidtvorennya_nas_m[0]+migrant_m_years[0]
    mrest_last=pd.Series(m_years.iloc[-1]+m_years.iloc[-2]+\
                vidtvorennya_nas_m.iloc[-1]+\
                migrant_m_years.iloc[-1])
    mrest=mrest4.append(mrest_last).reset_index(drop=True)
    mrest.index=mrest.index+1
    rest_m=pd.concat([mrest, pd.Series(mrest_first)])
    male_next=rest_m.sort_index()
    pop_next=female_next+male_next
    pop_next_total=pop_next.sum()
    
     
    
    return female_next,male_next,pop_next_total,vidtvorennya_nas_f       
   

In [8]:
def prognos(year,f_years,m_years,kfert):
    z=[]
    ye=[]
    pop_total=f_years.sum()+m_years.sum()
    for i in range(len(kfert)):
        fy=model(f_years,m_years,pop_total,kfert[i],0.136060581939936)[0]
        my=model(f_years,m_years,pop_total,kfert[i],0.136060581939936)[1]
        pt=model(f_years,m_years,pop_total,kfert[i],0.136060581939936)[2]
        z.append(model(f_years,m_years,pop_total,kfert[i],0.136060581939936)[2])
        f_years=fy
        m_years=my
        pop_total=pt
        ye.append(year+1+i)
        
    return pd.DataFrame(z,ye)

In [9]:
f=pop2017[' жінки'].reset_index(drop=True)
m=pop2017[' чоловіки'].reset_index(drop=True)

kfert_good=np.insert(np.repeat(95,40),[0],[46,60,65,70,75,80,85,90])
kfert_bad=np.repeat(46,37)

prognos(2017,f,m,np.repeat(38,100)).iloc[-1,:]

0   20434948.95
Name: 2117, dtype: float64

In [10]:
#исторические данные
#http://database.ukrcensus.gov.ua/MULT/Dialog/statfile_c_files/pasport.files/pasport/00_uk.htm#0301
#http://database.ukrcensus.gov.ua/PXWEB2007/ukr/publ_new1/2018/zb_dy_2017.pdf

pomerlyh=pd.Series([754460,706739,698235,664588,663139,662368,632296,594796,583631,574123],\
                   index=list(np.arange(2008,2018)),name='Померлих')
pomer_na_1000_nas=pd.Series([16.3,15.3,15.2,14.5,14.5,14.6,14.7,14.9,14.7,14.5],\
                            index=list(np.arange(2008,2018)),name='Померлих на 1000 населення')
zhivonar=pd.Series([510589,512525,497689,502595,520705,503657,465882,411781,397037,363987],\
                   index=list(np.arange(2008,2018)),name='Живонароджених')
zhivonar_na_1000_nas=pd.Series([11.0,11.1,10.8,11.0,11.4,11.1,10.8,10.7,10.3,9.4],\
                               index=list(np.arange(2008,2018)),name='Живонароджених на 1000 населення')
naselennya_vse=pd.Series([46372664,46143714,45962947,45778534,45633637,45553047,45426249,42929298,42760516,42584542,42386403],\
                         index=list(np.arange(2008,2019)),name='Населення України')
miske_nas=pd.Series([31668757,31587203,31524795,31441649,31380874,31378639,31336623,29673113,29584952,29482313,29370995],\
                    index=list(np.arange(2008,2019)),name='Міське населення')
silske_nas=pd.Series([14703907,14556511,14438152,14336885,14252763,14174408,14089626,13256185,13175564,13102229,13015408],\
                     index=list(np.arange(2008,2019)),name='Сільське населення')
zhivonar_m=pd.Series([263588,264844,256835,259433,268748,259738,240043,212846,205194,187528],\
                    index=list(np.arange(2008,2018)),name='Живонароджених чоловіків')
zhivonar_f=pd.Series([247001,247681,240854,243162,251957,243919,225839,198935,191843,176459],\
                    index=list(np.arange(2008,2018)),name='Живонароджених жінок')
mertvonar_all=pd.Series([3416,3351,3300,3158,3230,3166,2820,2437,2244,2136],\
                       index=list(np.arange(2008,2018)),name='Мертвонароджених')
mertvonar_m=pd.Series([1801,1821,1719,1649,1734,1683,1502,1322,1159,1090],\
                     index=list(np.arange(2008,2018)),name='Мертвонароджених чоловіків')

mertvonar_f=pd.Series([1615,1530,1581,1509,1496,1483,1318,1115,1085,1046],\
                     index=list(np.arange(2008,2018)),name='Мертвонароджених жінок')                    
                   



In [11]:
#https://ukrstat.org/uk/operativ/operativ2018/ds/mr/mr_u/mr1218_u.html
#Міграційний приріст, скорочення (–)
#[2011,2012,2013,2014,2015,2016,2017]
migraciya=pd.Series([17096,61844,31913,22592,14233,10620,11997],\
                    index=list(np.arange(2011,2018)),name='Міграційний приріст, скорочення (–)')


In [59]:
range61=np.arange(61,81)
range61_full=pop.loc[pop['вік'].isin(range61)].groupby('рік').sum()
naselennya_vid_61=range61_full['Обидві статі']/1000000
naselennya_vid_61.name='Населення від 61 року в Україні,млн.осіб'

In [60]:
chastka_61=naselennya_vid_61/nas_vse_mln*100
chastka_61.name='частка населення від 61 року від населення в Україні,%'

In [55]:
range_m18_60=np.arange(18,61)
range_f50_60=np.arange(50,61)
range_m18_60_full=pop.loc[pop['вік'].isin(range_m18_60)].groupby('рік').sum()
range_f50_60_full=pop.loc[pop['вік'].isin(range_f50_60)].groupby('рік').sum()
female_male=(range_m18_60_full[' чоловіки']+range_f50_60_full[' жінки'])/1000000
female_male.name='Чоловіки(18-60) та жінки(50-60) в Україні,млн.осіб'

In [57]:
chastka_fm=female_male/nas_vse_mln*100
chastka_fm.name='частка чоловіків (18-60) та жінок (50-60) від населення України, %'

In [43]:
range_ditei=np.arange(0,18)
ditei=pop.loc[pop['вік'].isin(range_ditei)].groupby('рік').sum()
ditei=ditei['Обидві статі']/1000000
ditei.name='Дітей(0-17)в Україні, млн.осіб'


In [45]:
chastka_ditei=ditei/nas_vse_mln*100
chastka_ditei.name='частка дітей від населення України'

In [39]:
range_fert=np.arange(18,50)
fert=pop.loc[pop['вік'].isin(range_fert)].groupby('рік').sum()
fert=fert[' жінки']/1000000
fert.name='Жінок дітородного віку(18-49) в Україні,млн.осіб'


In [41]:
chastka_fert=fert/nas_vse_mln*100
chastka_fert.name='частка жінок дітородного віку від населення України,%'

In [47]:
ditei_na_zhinku=ditei/fert
ditei_na_zhinku.name='Дітей на одну жінку дітородного віку в Україні,осіб'

In [49]:
zhivonar_mln=zhivonar/1000000
zhivonar_mln.name='Народилося в Україні за рік, млн.осіб'

In [53]:
narodylos_na_1000_zhinok=zhivonar_mln/fert*1000
narodylos_na_1000_zhinok.name='Народилося в Україні на 1000 жінок дітородного віку,осіб'

In [22]:
world_pop=pd.read_csv("D:/Users/o.lukasevych/Documents/Python_Scripts/Ukr_budget/Shiny_app/population/data/WPP2017_TotalPopulationBySex.csv")

In [23]:
subset1=world_pop.loc[(world_pop.Location=='World')&(world_pop.Variant=='Medium')]
subset2=subset1[['Time','PopTotal']]
world_pop_prognos=subset2.set_index('Time').iloc[:,0]

In [24]:
world_pop_slice=world_pop_prognos.loc[2011:2018]/1000
world_pop_slice.name='Населення світу,млн.осіб'

In [25]:
world_pop_change=world_pop_slice.pct_change()*100
world_pop_change.name='Темп росту населення світу в рік,%'

In [26]:
gdp=pd.read_excel('D:/Users/o.lukasevych/Documents/Python_Scripts/Ukr_budget/excel/gdp.xlsx')

In [27]:
gdp_world=gdp.iloc[1]['2011':'2017']/1000000000
gdp_world.index=gdp_world.index.astype('int')
gdp_world.name='ВВП світу,млрд.дол.США'

gdp_ukr=gdp.iloc[0]['2011':'2017']
gdp_ukr_mlrd=gdp_ukr/1000000000
gdp_ukr_mlrd.name='ВВП України,млрд.дол.США'
gdp_ukr_mlrd.index=gdp_ukr_mlrd.index.astype('int')

In [28]:
gdp_world_change=gdp_world.pct_change()*100
gdp_world_change.name='Темп росту ВВП світу в рік,%'
gdp_world_change.index=gdp_world_change.index.astype('int')

In [29]:
nas_vse_mln=naselennya_vse/1000000
nas_vse_mln.name='Населення України,млн.осіб'
gdpukr_na_osobu=gdp_ukr_mlrd/nas_vse_mln
gdpukr_na_osobu.name='ВВП України на одну особу,тис.дол.США'
gdpworld_na_osobu=gdp_world/(world_pop_slice)
gdpworld_na_osobu.name='ВВП світу на особу, тис.дол.США'

In [30]:
chastka_nasukr=nas_vse_mln/world_pop_slice*100
chastka_nasukr.name='частка населення України від населення світу,%'

In [31]:
chastka_gdp_na_osobu=gdpukr_na_osobu/gdpworld_na_osobu*100
chastka_gdp_na_osobu.name='частка ВВП України від ВВП світу на одну особу,%'

In [32]:
miske_nas_mln=miske_nas/1000000
miske_nas_mln.name='у т.ч. міського населення України,млн.осіб'

In [33]:
chastka_miske_nas=miske_nas/naselennya_vse*100
chastka_miske_nas.name='частка міського населення від населення України,%'

In [34]:
saldo_mifrant=migraciya/1000000
saldo_mifrant.name='Сальдо міжнародних мігрантів в Україні,млн.осіб'
chastka_migr=saldo_mifrant/nas_vse_mln*100
chastka_migr.name='частка сальдо міжнародних мігрантів від населення України,%'

In [64]:
pomerlyh_mln=pomerlyh/1000000
pomerlyh_mln.name='Померлих за рік в Україні,млн.осіб'

In [66]:
pomer_na_1000_nas.name='Померлих на 1000 жителів України,осіб'

In [68]:
historical=pd.concat([world_pop_slice,world_pop_change,gdp_world,gdpworld_na_osobu,gdp_world_change,\
          nas_vse_mln,chastka_nasukr,gdp_ukr_mlrd,gdpukr_na_osobu,chastka_gdp_na_osobu,\
          miske_nas_mln,chastka_miske_nas,saldo_mifrant,chastka_migr,\
          fert,chastka_fert,ditei,chastka_ditei,ditei_na_zhinku,zhivonar_mln,\
          narodylos_na_1000_zhinok,female_male,chastka_fm,naselennya_vid_61,\
          chastka_61,pomerlyh_mln,pomer_na_1000_nas],axis=1).T

In [82]:
historical.loc[:,2012:2017].to_excel('historical2012_2017_slide5.xlsx')